## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/TextMining_WordEmbedding

[Errno 2] No such file or directory: 'drive/MyDrive/TextMining_WordEmbedding'
/content


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.c

In [5]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [6]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [8]:
# data 불러오기
data_dir = os.path.join('/content/drive/MyDrive/DSL/TextMining_WordEmbedding/data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [9]:
corpus[:50]

['‘융합’의 바람은 기계공학에서도 거세게 불고 있다',
 '나노·바이오·에너지가 공학과 만나 새로운 학문이 속속 등장하고 있으며 기계공학자들이 이들 신생 연구 분야에 뛰어들고 있다',
 '조맹효 서울대 기계항공공학부 교수는 “고전적 기계공학의 외연을 넓히려는 시도가 지난 10여년 동안 급박하게 진행됐다”며 “융합·협력 연구는 이미 세계적인 추세”라고 말했다',
 '조 교수의 이번 연구 성과도 기계공학에 재료과학, 역학적 시각을 더해 이뤄낸 것이다',
 '학문 간 융합은 시선을 돌리는 이치와 같다',
 '각도를 달리 보면 난제를 해결할 방법과 의도치 않게 맞닥뜨릴 수 있다는 것이 조 교수의 설명이다',
 '기계공학뿐만이 아니다',
 '전기나 토목·건축 등 주로 물리학을 기반으로 하는 학문에서도 변화의 조짐이 보인다',
 '해당 학문들은 물리학의 특성상 방정식을 통해 데이터(해)를 구하는 것이 특성인데 반대로 축적된 데이터로 공식을 만드는 시도가 이뤄지는 것이다',
 '전산학의 영역이라고 여겨졌던 데이터 과학과 물리학이 만나기 시작한 것이다',
 '데이터 활용은 필연적으로 컴퓨터가 데이터 분석으로 스스로 학습해 궁극적으로 수많은 데이터를 분석하도록 하는 딥러닝 인공지능(AI)과의 접점을 형성한다',
 '조 교수는 “예를 들어 미국 항공우주국(NASA)에는 그동안의 비행 관련 실험이 무수히 쌓여 있는데 이를 데이터베이스화해서 데이터를 추출하고 의미 있는 정보로 만드는 ‘역순’으로 계산식을 도출하는 형태”라고 설명했다',
 '데이터를 분석해 모형을 설계하는 것도 가능하다',
 '하지만 융합은 말처럼 쉽지 않다',
 '‘다학제’ 연구를 하려면 여러 전문 분야를 알아야 한다',
 '그래야 연결 고리를 찾을 수 있기 때문이다',
 '또 학문·분야 간 접점을 찾으려는 노력이 중요하다',
 '조 교수는 새로운 시도를 강조하지만 연구 방법은 우직해야 한다고 믿는다',
 '“연구를 진행하면서 연구자 본인이 자신의 방법을 의심하기 시작하면 연구는 미궁으로 빠집니다',
 '연구의 

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]
"는 누군가의 말을 인용했음을 알기 위해 유지
그 외 문장부호, 조사, 접미사 등은 실질형태소가 아니므로 제거
norm을 통해 같은 의미를 같게 해석

In [47]:
okt = Okt()

In [48]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence,stop=['Josa','Suffix','Punctuation']):
    tokenized=[]
    for i, j in okt.pos(sentence, stem=False, norm=True):
      if j in stop:
        if j== '\"':
          tokenized.append(i)
        continue
      tokenized.append(i)

    return tokenized

In [49]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [50]:
print(corpus_tokenized)

[['‘', '융합', '의', '바람', '기계공학', '거세', '불', '있다'], ['나노', '바이오', '에너지', '공학', '만나', '새로운', '학문', '속속', '등장', '있으며', '기계공학자', '이', '신생', '연구', '분야', '뛰어들고', '있다'], ['조맹', '효', '서울대', '기계', '항공공학', '부', '교수', '“', '고', '전적', '기계공학', '외연', '넓히려는', '시도', '지난', '10', '여', '년', '동안', '급박하게', '진행', '됐다', '”', '며', '“', '융합', '협력', '연구', '이미', '세계', '추세', '”', '말', '했다'], ['조', '교수', '이번', '연구', '성과', '기계공학', '재료과학', '역학', '시각', '더해', '이뤄', '낸', '것'], ['학문', '간', '융합', '시선', '돌리는', '이치', '같다'], ['각도', '달리', '보면', '난제', '해결', '할', '방법', '의도', '치', '않게', '맞닥뜨릴', '수', '있다는', '것', '조', '교수', '설명'], ['기계공학', '아니다'], ['전기', '토목', '건축', '등', '주로', '물리학', '기반', '하는', '학문', '변화', '조짐', '보인다'], ['해당', '학문', '물리학', '특성', '방정식', '통해', '데이터', '해', '를', '구', '하는', '것', '특성', '반대', '축적', '된', '데이터', '공식', '만드는', '시도', '이뤄지는', '것'], ['전산학', '영역', '여겨졌던', '데이터', '과학', '물리학', '만나기', '시작', '것'], ['데이터', '활용', '필연', '컴퓨터', '데이터', '분석', '스스로', '학습', '해', '궁극', '수많은', '데이터', '분석', '하도록', '하는', '딥', '러닝', '인공', '지능',

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [95]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size=200, min_count=0,window=2,hs=1,negative=2)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 4860


In [96]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 10)

(166452, 177590)

In [102]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용
#print(model_no_pretrained.wv.vocab.keys())
for i in ['학문','교수','교통사고']:
  print(i,":",model_no_pretrained.wv.most_similar(i))

학문 : [('촉진', 0.9607917070388794), ('점검', 0.9604479074478149), ('어른', 0.9585084319114685), ('유실', 0.9573928117752075), ('소', 0.9525214433670044), ('개구리', 0.9506933689117432), ('사고', 0.944715678691864), ('전지', 0.9444553852081299), ('증명', 0.9410218000411987), ('쓰이지만', 0.9381349086761475)]
교수 : [('저하', 0.9000194072723389), ('중인', 0.8913394212722778), ('연례', 0.8769382834434509), ('연', 0.8705677390098572), ('이리저리', 0.8250399231910706), ('유전자', 0.8152942061424255), ('증상', 0.8145490884780884), ('고난', 0.8036801815032959), ('포스텍', 0.7857762575149536), ('탄생', 0.7853489518165588)]
교통사고 : [('고리', 0.9734041094779968), ('시키지', 0.9712957143783569), ('커졌고', 0.9708104133605957), ('내비게이션', 0.9699233770370483), ('실험동물', 0.9683812856674194), ('원일', 0.9679611921310425), ('바뀌면서', 0.9674968123435974), ('끈기', 0.967430830001831), ('전선', 0.9673164486885071), ('채팅봇', 0.9672948718070984)]


In [66]:
import os


/content


In [75]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
import gensim
pretrained = gensim.models.Word2Vec.load('/content/drive/MyDrive/DSL/TextMining_WordEmbedding/ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
pretrained.wv.save_word2vec_format('ko.bin.gz', binary = False)
model_pretrained = Word2Vec(size=200,min_count=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 4860


In [76]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 32372


In [77]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 10)

(174482, 177590)

In [88]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용
#model_pretrained.wv.vocab.keys()
for i in ['학문','교수','교통사고','수공업','공휴일','정보원','대마초']:
  print(i,":",model_pretrained.wv.most_similar(i))


학문 : [('성리학', 0.7003298997879028), ('학문적', 0.6835934519767761), ('학풍', 0.6695623397827148), ('경학', 0.6455956697463989), ('도학', 0.6370865106582642), ('의술', 0.6361593008041382), ('인문학', 0.63337242603302), ('주자학', 0.6331478953361511), ('철학', 0.627078652381897), ('사색', 0.6187766790390015)]
교수 : [('부교수', 0.7260501384735107), ('배호', 0.7224751710891724), ('강사', 0.689142107963562), ('조교수', 0.680406928062439), ('강정구', 0.680322527885437), ('박사', 0.6780283451080322), ('단과대학', 0.6674095392227173), ('먼은', 0.6583782434463501), ('학장', 0.6548829078674316), ('선웅', 0.65422523021698)]
교통사고 : [('뇌출혈', 0.672704815864563), ('수모', 0.6481466293334961), ('사고', 0.6318626999855042), ('피격', 0.6101049780845642), ('폭행', 0.6037010550498962), ('질식사', 0.6003941297531128), ('구타', 0.5976691842079163), ('조난', 0.5841835737228394), ('콜레라', 0.5811874270439148), ('화재', 0.5728487968444824)]
수공업 : [('가내', 0.7762992978096008), ('제조업', 0.7673743963241577), ('목축업', 0.684337854385376), ('경공업', 0.682089626789093), ('수산업', 0.6810650

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?  
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요! 차이가 많았다. 

[여러분의 설명]
no...  
Yess 일단, 사전학습 없이 진행한 단어들은 교통사고, 실험동물처럼 아무런 관련 없는 단어들이 similar하다고 나오는 등의 오류가 대부분이었다. 문장 자체가 뉴스 기사여서 딱히 관련이 없는 단어들이 함께 나열될 때가 많은데 이로 인해 오차가 발생하지 않았나 추측해본다. 

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [103]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

4860

In [105]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i] )

In [111]:
pca = PCA(random_state = 0).fit_transform(vector)
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:, :2]

In [109]:
# 시각화해보세요!

data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning: plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin

  DeprecationWarning,


In [113]:

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    mode='markers',
    marker=dict(
        size=5,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [116]:
  # t-SNE
for i in [5,10,15,25,35,45,60]:
    tsne = TSNE(n_components = 2, random_state = 0, perplexity = i, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)[:, :2]
    # 시각화

    data = go.Scatter(
        x=tsne[:,0],
        y=tsne[:,1],
        mode='markers',
        marker=dict(
            size=5,
            colorscale='Jet',
            line=dict(
                width=0.0
            ),
            opacity=0.8
        )
    )

    layout = go.Layout(
        autosize=False,
        width=1000,
        height=1000,
        margin=go.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
    )

    fig = go.Figure(data=[data], layout=layout)
    fig.show()


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!